In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import warnings

from training_ml_control.shortest_path_problem import (
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
)

warnings.simplefilter("ignore", UserWarning)

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

## Dynamic Programming

Dynamic programming (DP) is a method that in general solves optimization problems that involve making a sequence of decisions by determining, for each decision, subproblems that can be solved in like fashion, such that an optimal
solution of the original problem can be found from optimal solutions of sub-
problems. This method is based on Bellman’s Principle of Optimality, which
he phrased as follows:

> An optimal policy has the property that whatever the initial state and
initial decision are, the remaining decisions must constitute an optimal
policy with regard to the state resulting from the first decision.

:::{figure} _static/images/20_optimality_principle.png
:width: 50%
:align: center
Schematic illustration of the principle of optimality. The tail $\{u_k^∗, \dots, u_{N-1}^*\}$ of an optimal sequence $\{u_0^∗, \dots, u_{N-1}^*\}$ is optimal for the tail subproblem that starts at the state $x_k^*$ of the optimal state trajectory.
:::

Dynamic Programming is a very general solution method for problems which have two properties:

- Optimal substructure (Principle of optimality applies)
  - Optimal solution can be decomposed into subproblems, e.g., shortest path
- Overlapping subproblems
  - Subproblems recur many times
  - Solutions can be cached and reused

Dynamic programming is used across a wide variety of domains, e.g.

- Scheduling algorithms
- Graph algorithms (e.g., shortest path algorithms)
- Graphical models in ML (e.g., Viterbi algorithm)
- Bioinformatics (e.g., Sequence alignment, Protein folding) 

We define the **cost-to-go function** (also known as **optimal value function**) as (for the sake of simplicity we will focus on the discrete-time case):

$$
V_0(x_0) := \min_{u \in \mathbf{U}} J_0(x_0, u)
$$

An admissible control sequence $u^*$ is called optimal, if

$$
V_0(x_0) = J_0(x_0, u^*)
$$

For any feasible $x_0 \in \mathbf{X}$ the optimal value function satisfies

$$
V_k(x_0) = \displaystyle \min_{u \in \mathbf{U}} g_{k}(x_0, u) + V_{k+1}(f(x_0, u))
$$

Moreover, if $u^*$ is an optimal control, then

$$
V_0(x_0) = g_{k}(x_0, u) + V_{k+1}(f(x_0, u))
$$

and

$$
V_0(x_0) = J_0(x_0, u^*)
$$

###  DP Algorithm

For every initial state $x_0$, the optimal cost is equal to $V_N(x_0)$, given by the last step of the following algorithm, which proceeds backward in time from stage $N-1$ to stage $0$:

- Start with $V_N(x_N) = g_N(x_N)$
- then for $k = \{0, \dots , N - 1\}$, let:
 
  $$
  V_{k}(x_{k}) = \displaystyle \min_{u \in \mathbf{U}} \left\{ g_{k}(x_{k}, u_{k}) + V_{k+1}(f(x_{k}, u_{k})) \right\}
  $$
  
Once the functions $V_0, \dots , V_N$ have been obtained, we can use a forward algorithm to construct an optimal control sequence $\{u_0^*, \dots, u_{N-1}^*\}$ and corresponding state trajectory $\{x_1^∗, \dots, x_{N}^*\}$ for the given initial state $x_0$ .

:::{figure} _static/images/20_dynamic_programming.png
:width: 60%
Illustration of the DP algorithm. The tail subproblem that starts at $x_k$ at time $k$ minimizes over
$\{u_k , \dots , u_{N-1}\}$ the "cost-to-go" from $k$ to $N$.
:::

````{exercise-start} Shortest Path
:label: shortest-path
````

In [ ]:
G = create_shortest_path_graph()
plot_shortest_path_graph(G)

We wish to travel from node A to node G at minimum cost. If the cost represents time then we want to find the shortest path from A to G.

- Arrows (edges) indicate the possible movements.
- Numbers on edges indicate the cost of moving along an edge.

Use Dynamic Programming to solve this problem.

```{tip} Hint
:class: dropdown
Determine all possible paths first and then compute the cost-to-go at each node.

You can call `plot_all_paths_graph(G)
```

````{exercise-end}
````

````{solution-start} shortest-path
````

In [ ]:
# Your Solution Here

```{solution-end}
```

::::{solution} shortest-path
:class: dropdown

:::{code-cell} python3
plot_all_paths_graph(G)
:::

:::{tip} Solving for the shortest path
Each node in this new graph represents a state. We will start from the tail (the last states) and compute recursively the cost for each state transition.

We start by determining the cost-to-go for all positions.

Let $l(n_1, n_2)$ the cost of going from node $n_1$ to $n_2$ and $V(n)$ be the cost-to-go from node $n$.

$$
\begin{array}{lll}
V(\text{ABDF}) &= g(\text{ABDF}, \text{ABDFG}) &= 1\\
V(\text{ABE}) &= g(\text{ABE}, \text{ABEG}) &= 4\\
V(\text{ACF}) &= g(\text{ACF}, \text{ACFG}) &= 1\\
V(\text{ADF}) &= g(\text{ADF}, \text{ADFG}) &= 1\\
\end{array}
$$

$$
\begin{array}{lll}
V(\text{ABD}) &= \min \left[ g(\text{ABD}, \text{ABDG}), g(\text{ABD}, \text{ABDF}) + V(\text{ABDF}) \right]
&= \min \left[ 8, 5 + 1 \right] &= 6
\\
V(\text{AB}) &= \min \left[ g(\text{AB}, \text{ABD}) + V(\text{ABD}), g(\text{AB}, \text{ABE}) + V(\text{ABE}) \right]
&= \min \left[ 9 + 6, 1 + 4 \right] &= 5
\\
V(\text{AC}) &= g(\text{AC}, \text{ACF}) + V(\text{ACF}) &= 2 + 1 &= 3
\\
V(\text{AD}) &= \min \left[ g(\text{AD}, \text{ADF}) + V(\text{ADF}), g(\text{AD}, \text{ADG})) \right]
&= \min \left[ 5 + 1, 8 \right] &= 6
\\
\end{array}
$$

$$
\begin{array}{lll}
V(\text{A}) &= \min \left[
g(\text{A}, \text{AB}) + V(\text{AB}), g(\text{A}, \text{AC}) + V(\text{AC}), g(\text{A}, \text{AD}) + V(\text{AD})
\right]
&= \min \left[ 1 + 5, 5 + 3, 3 + 6 \right] &= 6
\\
\end{array}
$$

The shortest-path is ABEG.
:::

:::{code-cell} python3
plot_all_paths_graph(G, show_solution=True)
:::

::::

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::